import pandas as pd
import numpy as np
from numpy.random import randint
from numpy.random import rand

In [124]:
df = pd.DataFrame({
'item':["compass","hatchet",'matches','tarpaulin','telescope','cylinder', 'rikkaluma'],
"weight":[0.5,1.5,0.4,1,1.1, 1.6,0.8],
'value':[6,5,4,4,3,4,1]
})
df

,item,weight,value
0,compass,0.5,6
1,hatchet,1.5,5
2,matches,0.4,4
3,tarpaulin,1.0,4
4,telescope,1.1,3
5,cylinder,1.6,4
6,rikkaluma,0.8,1


In [125]:
# objective function
from cmath import inf

def knapsack(x, value, s_t_):

	if sum(x * s_t_) <= 5:
		return sum(x * value)
	else:
		return -inf

In [126]:

# tournament selection
def selection(pop, scores, k=3):
	# first random selection
	selection_ix = randint(len(pop))
	for ix in randint(0, len(pop), k-1):
		# check if better (e.g. perform a tournament)
		if scores[ix] > scores[selection_ix] :

			selection_ix = ix
	return pop[selection_ix]

# crossover two parents to create two children
def crossover(p1, p2, r_cross):
	# children are copies of parents by default
	c1, c2 = p1.copy(), p2.copy()
	# check for recombination
	if rand() < r_cross:
		# select crossover point that is not on the end of the string
		pt = randint(1, len(p1)-2)
		# perform crossover
		c1 = np.append(p1[:pt] , p2[pt:])
		c2 = np.append(p2[:pt] , p1[pt:])
	return [c1, c2]

# mutation operator
def mutation(bitstring, r_mut):
	for i in range(len(bitstring)):
		# check for a mutation
		if rand() < r_mut:
			# flip the bit
			bitstring[i] = 1 - bitstring[i]

# genetic algorithm
def genetic_algorithm(objective, n_bits, n_iter, n_pop, r_cross, r_mut):
	# initial population of random bitstring
	pop = [randint(0, 2, n_bits) for _ in range(n_pop)]
	# keep track of best solution
	best, best_eval = 0, objective(pop[0], value, s_t_)
	# enumerate generations
	for gen in range(n_iter):
		# evaluate all candidates in the population
		scores = [objective(c, value, s_t_) for c in pop]
		# check for new best solution
		for i in range(n_pop):
			if scores[i] > best_eval:
				best, best_eval = pop[i], scores[i]
				print(">%d, new best f(%s) = %.3f" % (gen,  pop[i], scores[i]))
		# select parents
		selected = [selection(pop, scores) for _ in range(n_pop)]
		# create the next generation
		children = list()
		for i in range(0, n_pop, 2):
			# get selected parents in pairs
			p1, p2 = selected[i], selected[i+1]
			# crossover and mutation
			for c in crossover(p1, p2, r_cross):
				# mutation
				mutation(c, r_mut)
				# store for next generation
				children.append(c)
		# replace population
		pop = children
	return [best, best_eval]

# define the total iterations
n_iter = 100
# bits
n_bits = len(df['item'])
#selected variable for objective function
value = df['value'] 
#constraints/ subject to
s_t_ = df['weight']
# define the population size
n_pop = 100
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / float(n_bits)
# perform the genetic algorithm search
best, score = genetic_algorithm(knapsack, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
print('f(%s) = %f' % (best, score))

>0, new best f([1 1 1 1 1 0 0]) = 22.000
>0, new best f([1 1 1 1 0 1 0]) = 23.000
Done!
f([1 1 1 1 0 1 0]) = 23.000000
